# Module 2: Knowledge Graph Construction & Embedding

Welcome to Module 2! In the previous module, we successfully extracted structured data from our raw contract files into `contract_data.json`. Now, we'll complete the **Ingestion Pipeline** by taking that structured data and using it to build our AGL Knowledge Graph.

**Our Mission:**
1. Connect to a Neo4j database.
2. Generate vector embeddings for our contract summaries.
3. Ingest both the structured data and the embeddings into Neo4j to create a fully queryable knowledge graph.

## 1. Setup and Dependencies

Let's start by installing the required libraries. We need `langchain-neo4j` to interact with our graph database and `langchain-google-genai` for creating embeddings.

In [1]:
!pip install -qU langchain-neo4j langchain-google-genai python-dotenv

In [2]:
import os
import json
from dotenv import load_dotenv
from google.colab import userdata

from langchain_neo4j import Neo4jGraph
from langchain_google_genai import GoogleGenerativeAIEmbeddings

from google.colab import drive
import json


## 2. Configure Environment Variables

This module requires two sets of credentials:
1.  **Google API Key:** To use the embedding model.
2.  **Neo4j Database Credentials:** To connect to our graph database.

**Action:** You can get free Neo4j AuraDB credentials from [Neo4j's website](https://neo4j.com/cloud/platform/aura-graph-database/).

In [3]:
# Define required environment variables
required_vars = ["GOOGLE_API_KEY", "NEO4J_URI", "NEO4J_USERNAME", "NEO4J_PASSWORD", "NEO4J_DATABASE"]

# Set environment variables with validation
missing_vars = []
for var in required_vars:
    value = userdata.get(var)
    if value:
        os.environ[var] = value
        print(f"✅ {var}: Set successfully")
    else:
        missing_vars.append(var)
        print(f"❌ {var}: Missing or empty")

# Check if all required variables are set
if missing_vars:
    print(f"\n🚨 Error: Missing required environment variables: {', '.join(missing_vars)}")
    print("Please ensure all secrets are properly configured in Colab.")
    raise ValueError(f"Missing environment variables: {missing_vars}")
else:
    print(f"\n🎉 Successfully loaded all {len(required_vars)} required environment variables!")

# Optional: Verify API key format (basic validation)
if os.environ.get("GOOGLE_API_KEY"):
    api_key = os.environ["GOOGLE_API_KEY"]
    if len(api_key) < 20:  # Basic length check
        print("⚠️  Warning: Google API key seems unusually short")
    else:
        print("✅ Google API key format looks valid")

✅ GOOGLE_API_KEY: Set successfully
✅ NEO4J_URI: Set successfully
✅ NEO4J_USERNAME: Set successfully
✅ NEO4J_PASSWORD: Set successfully
✅ NEO4J_DATABASE: Set successfully

🎉 Successfully loaded all 5 required environment variables!
✅ Google API key format looks valid


## 3. Connect to Neo4j and Clear Existing Data

Let's establish a connection to our database. For a clean start, we'll also run a query to delete any existing data in the graph. This ensures our ingestion pipeline is repeatable.

In [5]:
import time
from neo4j.exceptions import ServiceUnavailable, TransientError

max_connection_retries = 3
connection_delay = 2

for attempt in range(max_connection_retries):
    try:
        print(f"Attempting to connect to Neo4j (attempt {attempt + 1}/{max_connection_retries})...")

        graph = Neo4jGraph(
            url=os.environ["NEO4J_URI"],
            username=os.environ["NEO4J_USERNAME"],
            password=os.environ["NEO4J_PASSWORD"],
            database=os.environ["NEO4J_DATABASE"]
        )

        # Test the connection with a simple query
        graph.query("RETURN 1 as test")
        print("✅ Neo4j connection established successfully")

        # Clear the database for a clean import
        print("🧹 Clearing existing data...")
        graph.query("MATCH (n) DETACH DELETE n")
        print("✅ Successfully connected to Neo4j and cleared existing data.")
        break

    except (ServiceUnavailable, TransientError, TimeoutError) as e:
        print(f"❌ Connection error on attempt {attempt + 1}: {e}")

        if attempt < max_connection_retries - 1:
            print(f"⏳ Waiting {connection_delay} seconds before retry...")
            time.sleep(connection_delay)
            connection_delay *= 2  # Exponential backoff
        else:
            print(f"💥 Failed to connect to Neo4j after {max_connection_retries} attempts")
            raise e

    except Exception as e:
        print(f"❌ Unexpected error connecting to Neo4j: {e}")
        raise e

Attempting to connect to Neo4j (attempt 1/3)...
✅ Neo4j connection established successfully
🧹 Clearing existing data...
✅ Successfully connected to Neo4j and cleared existing data.


## 4. Load Extracted Contract Data

Here we load the `contract_data.json` file generated in Module 1.

This file contains structured contract information

We'll reference this data for further processing in the pipeline.

In [6]:
# Load the structured contract data extracted in Module 1.
# The variable 'contract_results' will hold a list of dictionaries,
# where each dictionary contains information about a single contract
# (such as summary, contract_type, parties, clauses, etc.).


# Mount Google Drive
drive.mount('/content/drive')

# Load the file (replace with your actual path)
contract_json_filename = "/content/drive/MyDrive/1. Learning/01_GenAI_Community_Work/02-GenAI Collab Hub/Cohort-2/Notebooks/contract_data.json"

with open(contract_json_filename, 'r') as file:
    contract_results = json.load(file)

print(f"Successfully loaded {len(contract_results)} records")
print(contract_results)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Successfully loaded 5 records
[{'summary': 'Reseller promotes and solicits commitments to buy company products in the territory. Contract is renewable for 1 year extension by amendment to this agreement. Either Party may terminate this agreement for non-cause with a sixty (60) day written notice.', 'contract_type': 'Reseller', 'parties': [{'name': 'Aperture Global Logistics', 'location': {'city': 'Anytown', 'state': 'Delaware', 'country': 'US'}, 'role': 'Reseller'}, {'name': 'LogiSync Solutions Inc.', 'location': {'city': 'Silicon Valley', 'state': 'CA', 'country': 'US'}, 'role': 'Company'}], 'effective_date': '2017-04-07', 'duration': 'P1Y', 'end_date': '2018-04-07', 'governing_law': {'city': None, 'state': 'Virginia', 'country': 'US'}, 'clauses': [{'summary': "Company shall indemnify, defend and hold Reseller harmless from and against any claim that the Com

## 5. Create Vector Embeddings

Here we generate vector embeddings for each contract summary using Google's Gemini text-embedding-004 model.

This code uses Gemini's embedding API, processes summaries in small batches to avoid timeouts, and implements retries for robustness.

Each embedding generated by the Gemini model is attached to its corresponding contract in the data.


In [7]:
import time
from google.api_core import retry

# if condition to check if 'contract_results' exists in the current local scope.
# This prevents errors if the contract data failed to load or was not defined.
if 'contract_results' in locals():
    # Initialize the Gemini embedding model
    embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
    # Extract summaries from each contract, defaulting to empty string if missing
    summaries = [el.get("summary", "") for el in contract_results]

    batch_size = 2  # Number of summaries to process in each batch
    embeddings_output = []  # List to store all embeddings

    print(f"Generating embeddings for {len(summaries)} summaries in batches of {batch_size}...")

    # Process summaries in batches
    for i in range(0, len(summaries), batch_size):
        batch = summaries[i:i + batch_size]
        batch_indices = list(range(i, min(i + batch_size, len(summaries))))
        print(f"Processing batch {i//batch_size + 1}/{(len(summaries) + batch_size - 1)//batch_size}: summaries {batch_indices}")

        max_retries = 3  # Maximum number of retry attempts for each batch
        for attempt in range(max_retries):
            try:
                # Attempt to generate embeddings for the current batch using Gemini
                batch_embeddings = embeddings.embed_documents(batch)
                embeddings_output.extend(batch_embeddings)
                print(f"✓ Successfully embedded batch {i//batch_size + 1}")
                break  # Exit retry loop on success
            except Exception as e:
                print(f"✗ Attempt {attempt + 1} failed for batch {i//batch_size + 1}: {e}")
                if attempt < max_retries - 1:
                    wait_time = (attempt + 1) * 2  # Exponential backoff for retries
                    print(f"Waiting {wait_time} seconds before retry...")
                    time.sleep(wait_time)
                else:
                    print(f"Failed to embed batch {i//batch_size + 1} after {max_retries} attempts")
                    raise e  # Raise exception if all retries fail

        # Optional: short pause between batches to avoid rate limits
        if i + batch_size < len(summaries):
            time.sleep(1)

    # Attach each Gemini embedding to its corresponding contract
    for i, contract in enumerate(contract_results):
        contract['embedding'] = embeddings_output[i]

Generating embeddings for 5 summaries in batches of 2...
Processing batch 1/3: summaries [0, 1]
✓ Successfully embedded batch 1
Processing batch 2/3: summaries [2, 3]
✓ Successfully embedded batch 2
Processing batch 3/3: summaries [4]
✓ Successfully embedded batch 3


## 6 Save Contract Embeddings to a json file

Here we prepare a list of contract file IDs and their embedding vectors for Neo4j import.
We then saves these embeddings to a JSON file for later use in the graph database.

In [8]:
params = []
for embedding, contract in zip(embeddings_output, contract_results):
    params.append({"file_id": contract["file_id"], "embedding": embedding})

with open("contract_embedding.json", "w") as json_file:
    json.dump(params, json_file, indent=4)

print(f"💾 Saved embeddings to contract_embedding.json")
print(f"Successfully generated embeddings for {len(embeddings_output)} contract summaries.")
print(f"Sample embedding vector length: {len(contract_results[0]['embedding'])}")

💾 Saved embeddings to contract_embedding.json
Successfully generated embeddings for 5 contract summaries.
Sample embedding vector length: 768


## 7. Define the Graph Ingestion Query (Cypher Query for Import)

Here we define the **Cypher query for importing contract data** into the Neo4j graph.

This is the core of our module: the Cypher query that will build our knowledge graph. The query uses `UNWIND` to process our list of contracts in a single, efficient transaction.

For each contract, it will:
1.  **MERGE** a `Contract` node, using `file_id` as a unique key.
2.  **SET** its properties (summary, type, dates, etc.).
3.  **MERGE** `Party` and `Location` nodes for all parties involved, creating relationships like `:PARTY_TO` and `:HAS_LOCATION`.
4.  **MERGE** `Clause` nodes and link them to the contract with a `:HAS_CLAUSE` relationship.
5.  **Set** the `embedding` vector property on the `Contract` node.

In [9]:
query = """UNWIND $data AS row
MERGE (c:Contract {file_id: row.file_id})
SET c.summary = row.summary,
    c.contract_type = row.contract_type,
    c.effective_date = date(row.effective_date),
    c.contract_scope = row.contract_scope,
    c.duration = row.duration,
    c.end_date = CASE WHEN row.end_date IS NOT NULL THEN date(row.end_date) ELSE NULL END,
    c.total_amount = row.total_amount
WITH c, row
CALL (c, row) {
    WITH c, row
    WHERE row.governing_law IS NOT NULL
    MERGE (c)-[:HAS_GOVERNING_LAW]->(l:Location)
    SET l += row.governing_law
}
FOREACH (party IN row.parties |
    MERGE (p:Party {name: party.name})
    MERGE (p)-[:HAS_LOCATION]->(pl:Location)
    SET pl += party.location
    MERGE (p)-[pr:PARTY_TO]->(c)
    SET pr.role = party.role
)
FOREACH (clause IN row.clauses |
    MERGE (c)-[:HAS_CLAUSE]->(cl:Clause {type: clause.clause_type})
    SET cl.summary = clause.summary
)
"""

## 8. Execute Cypher query

Here we execute the cypher query defined above to actually import all structured contract results.

1. First we load previously saved contract data from the JSON file into the results variable, allowing us to reuse the parsed contract objects without re-processing the original data source.

2. We create unique constraints in Neo4j to ensure data integrity:

- The first constraint enforces that each Contract node must have a unique file_id property (prevents duplicate contracts).

- The second constraint ensures each Party node has a unique name (prevents duplicate parties).

3. Then we execute the cypher query defined above to actually import all structured contract results.

In [10]:
# Create unique constraint for Contract nodes on file_id
graph.query("CREATE CONSTRAINT IF NOT EXISTS FOR (c:Contract) REQUIRE c.file_id IS UNIQUE;")
# Create unique constraint for Party nodes on name
graph.query("CREATE CONSTRAINT IF NOT EXISTS FOR (c:Party) REQUIRE c.name IS UNIQUE;")

# Execute Cypher query to import contract data
graph.query(query, {"data": contract_results})

[]

## 9. Add embedding vectors to contract nodes for semantic search.

In [11]:
with open("contract_embedding.json", "w") as json_file:
    json.dump(params, json_file, indent=4)

graph.query("""UNWIND $data AS row
MATCH (c:Contract {file_id:row.file_id})
CALL db.create.setNodeVectorProperty(c, 'embedding', row.embedding)""",
            {"data": params})

[]

## 10. Create Vector Index

- **Vector Index**: We'll create a vector index on the `embedding` property of `Contract` nodes. This is essential for enabling fast similarity searches in the next module.

In [12]:
graph.query("CREATE VECTOR INDEX contractSummary IF NOT EXISTS FOR (c:Contract) ON c.embedding")

[]

### Congratulations!

You have successfully completed the Ingestion Pipeline! We have now transformed raw text into a fully populated, indexed, and semantically enriched knowledge graph in Neo4j. This graph is now ready to be queried by our intelligent agent, which we will build in Module 3.